# UE mobility prediction

The UE connects via a DN to the closest application server. Next, it moves to the next edge site. To keep latency low after a gNB handover, the UE’s context should be relocated not only on the gNB, but also on the UPF and the application server.

If the network knows in advance that the UE will move to the target edge site, then some parts of the relocation procedure can be done before the actual gNB handover. In this way, the overall procedure is shortened, and the handover can be performed more smoothly.


<div>
<img src="../image.png" width="400">
</div>

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams["figure.figsize"] = (10, 5)

## Data analysis

The dataset in `trajectories.json.gz` is made of events carrying the following features:
* `currentEnb`: gNB the user has moved to 
* `eventId`: type of event
* `imsi`: IMSI (user identifier) the event belongs to. 
* `timestamp`: minutes starting from 0 at day 1 in the dataset. The dataset contains data from about four days and a half 
* `timeSlot`: a 15-minute slot generated from timestamp. There should be 96 timeslots in a day. `timeSlot` is restarted every day. 

## Exploratory Data Analysis

In [ ]:
df = pd.read_json('trajectories.json.gz', compression='gzip')
df.head()

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df[df['imsi'] == 2785554734]

In [ ]:
df['currentEnb'].value_counts().sort_index().plot(kind='bar')

In [ ]:
df['timeSlot'].value_counts().sort_index().plot(kind='bar')

In [ ]:
df['timestamp'].max()

In [ ]:
df['timestamp'].hist(bins=5)

### Additional data sources

For each gNodeB, `topology_neighbours.json` contains its closest neighbours.

In [ ]:
gnb_neigh = pd.read_json('topology_neighbours.json', orient='index')
gnb_neigh.reset_index(inplace=True)
gnb_neigh.columns = ['gnodeB', 'neigh_1', 'neigh_2', 'neigh_3', 'neigh_4']
gnb_neigh.head()

## Feature engineering

First, we select the first day as train set.

In [ ]:
day_1 = df.loc[(df['timestamp']>=0) & (df['timestamp']<1440)]
day_2 = df.loc[(df['timestamp']>=1440) & (df['timestamp']<2880)]

For each EMB event, we create a number of new features:
* `stay_time`: time until the same user moves to another gNodeB (and thus a new event is received).
* `transit_time`: time it took the user to move from the the previous gNodeB. For a same user that goes from gNodeB-a to gNodeB-b, `stay_time` in gNodeB-A is the same as `transit_time` in gNodeB-b
* `transition_slot`: bucketed `transit_time` considering that any time greater than 60 minutes is assigned 100 minutes.
* `time_of_day`: bucketed `timeSlot` (five periods of time in the day).

In [ ]:
grouped_df = day_1.set_index(['imsi', 'timestamp']).sort_index().reset_index()
grouped_df['next_timestamp'] = grouped_df.groupby(['imsi'])['timestamp'].shift(-1)
grouped_df['prev_timestamp'] = grouped_df.groupby(['imsi'])['timestamp'].shift(1)
grouped_df['stay_time'] = grouped_df['next_timestamp'] - grouped_df['timestamp']
grouped_df['transit_time'] = grouped_df['timestamp'] - grouped_df['prev_timestamp']

In [ ]:
grouped_df['stay_time'].hist(bins=128)

See the peak around 700 minutes, that could correspond to people remaining at work for such period of time. It's interesting to note than bucketing to create `transition_slot` takes the maximum value of its bucket but in the larger bucket, where the value is set to 100. That is, all values greater than 60 (one hour) are set to 100.

In [ ]:
grouped_df['transit_time'].max()

In [ ]:
max_transit_time = grouped_df['transit_time'].max()
grouped_df['transition_slot'] = pd.cut(grouped_df['transit_time'], 
                                       [0, 1, 5, 15, 30, 60, max_transit_time],
                                       labels=[1, 5, 15, 30, 60, 100])
grouped_df['transition_slot'].value_counts().sort_index()

In [ ]:
grouped_df['transition_slot'].value_counts().sort_index().plot(kind='bar')

Five equal bins are defined. However, as the upper limit is the maximum value in `timeSlot`, assignment has not much to do with the intended meaning of the bins.

In [ ]:
grouped_df['time_of_day'] = pd.cut(grouped_df['timeSlot'],
                                   bins=5,
                                   labels=['early_morn', 'morning', 'noon', 'evening', 'night'])

In [ ]:
grouped_df['time_of_day'].value_counts().reindex(index=['early_morn', 'morning', 'noon', 'evening', 'night'])

In [ ]:
grouped_df['time_of_day'].value_counts().reindex(index=['early_morn', 'morning', 'noon', 'evening', 'night']).plot(kind='bar')

`ue_home_df` records how long a given user is at a given gNodeB (from `stay_time`)

In [ ]:
ue_home_df = grouped_df.groupby(['imsi', 'currentEnb'])['stay_time'].sum().reset_index()
ue_home_df.head()

`home_df` records the gNodeB where each user stays for longer

In [ ]:
home_gnb = ue_home_df.loc[ue_home_df.groupby(['imsi'])['stay_time'].idxmax()][['imsi', 'currentEnb']]
home_gnb.columns = ['imsi', 'home_gnb']
home_gnb.head()

`ue_df` records the number of gNodeB's a given user is in for each time of the day

In [ ]:
ue_df = grouped_df.groupby(['imsi', 'time_of_day'])['currentEnb'].count().reset_index()
ue_df.head()

`ue_ctxt_df` contains the same information than `ue_df` but pivotted to show a user per row

In [ ]:
ue_ctxt_df = pd.pivot_table(ue_df, index='imsi', columns='time_of_day', values='currentEnb', fill_value=0)
ue_ctxt_df.head()

`seq_df` is an auxiliary dataframe that, for each user, provides its trajectory, the time where each transition happened, the bucketed time it took for each user to move from the previous gNodeB and the signal strength in each gNodeB

In [ ]:
seq_df = grouped_df.groupby(['imsi'])['currentEnb', 'timeSlot','transition_slot'] \
                   .agg(lambda x: list(x)).reset_index()

seq_df.head()

In [ ]:
def window_sequence(seq, window_size=5):
    """This function turns a sequence intro 
    a matrix where each row is a sequence of
    the previous 'windows_size' elements"""
    
    windows = []
    seq_len = len(seq)
    for i in range(len(seq) - window_size + 1):
        window = []
        for j in range(i, i + window_size):
            window.append(seq[j])
        windows.append(window)
    return windows

Each feature in `seq_df` creates a new feature where the sequences from the original feature is recorded (five elements in each sequence):
* `gnode_seq` records the sequences of each gNodeB in `currentEnb`.
* `time_seq` records the sequences of each timeslots in `timeSlot`.
* `trans_seq` records the sequences of each transition slots in `transition_slot`.

Additionally, we create a new feature with the number of transitions:
* `seq_len`

In [ ]:
seq_df['gnode_seq'] = seq_df['currentEnb'].apply(window_sequence).values
seq_df['time_seq'] = seq_df['timeSlot'].apply(window_sequence).values
seq_df['trans_seq'] = seq_df['transition_slot'].apply(window_sequence).values
seq_df['seq_len'] = seq_df['gnode_seq'].apply(lambda x: len(x))
seq_df

Next, we transform each sequence feature into a new dataframe

In [ ]:
enode_seq_list = [sequence for sequences in seq_df['gnode_seq'] for sequence in sequences]
time_seq_list = [sequence for sequences in seq_df['time_seq'] for sequence in sequences]
trans_seq_list = [sequence for sequences in seq_df['trans_seq'] for sequence in sequences]

enode_df = pd.DataFrame(enode_seq_list, columns=['gnode_1', 'gnode_2', 'gnode_3', 'gnode_4', 'target_gnb'])
time_df = pd.DataFrame(time_seq_list, columns=['time_1', 'time_2', 'time_3', 'time_4', 'target_time'])
trans_df = pd.DataFrame(trans_seq_list, columns=['trans_1', 'trans_2', 'trans_3', 'trans_4', 'target_trans_slot'])

In [ ]:
imsi_list = list()
for a, b in zip(seq_df['imsi'], seq_df['seq_len']):
    imsi_list.extend([a] * b)

Finally, we create the dataset that will be used for prediction. What we want to predict is what follows: given a detected transition to a given gNodeB, which will be the following gNodeB and when will it happen?

Thus, the targets are:
* `target_gnb`: Next gNodeB
* `target_trans_slot`: Time it will take to move to the next gNodeB.

The following features will be considered:
* `enode_1`: The fourth previous gNodeB in a given sequence. Depends on the current user's trajectory.
* `enode_2`: The third previous gNodeB in a given sequence. Depends on the current user's trajectory.
* `enode_3`: The second previous gNodeB in a given sequence. Depends on the current user's trajectory.
* `enode_4`: The first previous gNodeB in a given sequence. Depends on the current user's trajectory.
* `time_1`: The time where the user moved to the fourth previous gNodeB in a given sequence. Depends on the current user's trajectory.
* `time_2`: The time where the user moved to the third previous gNodeB in a given sequence. Depends on the current user's trajectory.
* `time_3`: The time where the user moved to the second previous gNodeB in a given sequence. Depends on the current user's trajectory.
* `time_4`: The time where the user moved to the first previous gNodeB in a given sequence. Depends on the current user's trajectory.
* `home_gnb`: The gNodeB where the user doing for a given trajectory stays more times within a day. It's is a historical data.
* `early_morn`: The number of gNodeB's where the user for a given trajectory stays in the early morning. It's is a historical data.
* `morning`: The number of gNodeB's where the user for a given trajectory stays in the morning. It's is a historical data.
* `noon`: The number of gNodeB's where the user for a given trajectory stays at noon. It's is a historical data.
* `evening`: The number of gNodeB's where the user for a given trajectory stays in the evening. It's is a historical data.
* `night`: The number of gNodeB's where the user for a given trajectory stays at night. It's is a historical data.
* `neigh_1`: One of the four closest gNodeB's to `enode_4`. It's is a static data.
* `neigh_2`: One of the four closest gNodeB's to `enode_4`. It's is a static data.
* `neigh_3`: One of the four closest gNodeB's to `enode_4`. It's is a static data.
* `neigh_4`: One of the four closest gNodeB's to `enode_4`. It's is a static data.

In [ ]:
#### CHANGES
X_train = pd.concat([enode_df, time_df], axis=1)
X_train['target_trans_slot'] = trans_df['target_trans_slot']
X_train['imsi'] = imsi_list
X_train

In [ ]:
X_train = X_train.merge(home_gnb, on='imsi').merge(ue_ctxt_df, on='imsi')

In [ ]:
X_train = X_train.merge(gnb_neigh, left_on='gnode_4', right_on='gnodeB', how='left')

In [ ]:
del X_train['gnodeB']

In [ ]:
X_train.dropna(inplace=True)
X_train

In [ ]:
y_train = X_train[['target_gnb', 'target_trans_slot']]
y_train

In [ ]:
to_drop_cols = ['target_gnb',
                'target_trans_slot',
                'target_time',
                'imsi'
               ]
X_train.drop(columns=to_drop_cols, axis=1, inplace=True)
X_train

## Model training

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=30,
                               min_samples_leaf=2,
                               min_samples_split=2,
                               max_depth=15,
                               oob_score=True,
                               random_state=44)
model.fit(X_train, y_train)

## Model evaluation

In [ ]:
importance = model.feature_importances_
for i, v in zip(X_train.columns, importance):
    print(f'Feature: {i}, Score: {v}')

In [ ]:
pd.Series(index=X_train.columns, data=importance).plot.bar()

In [ ]:
import numpy as np

def compute_accuracy(y, y_pred):
    y_pred = pd.DataFrame(y_pred, columns=['target_gnb', 'target_trans_slot'])
    gnb_pred_acc = (sum(y.target_gnb.values == y_pred.target_gnb.values) / len(y)) * 100
    timeslot_pred_acc = (sum(y.target_trans_slot.values == y_pred.target_trans_slot.values) / len(y)) * 100
    pred_accuracy = (sum((y.target_gnb.values == y_pred.target_gnb.values) &
                            (y.target_trans_slot.values == y_pred.target_trans_slot.values)) / len(y)) * 100
    return {
        "predictionAccuracy": pred_accuracy,
        "predictionTimeSlotAccuracy": timeslot_pred_acc,
        "predictionGnodeBAccuracy": gnb_pred_acc
    }

def topk_accuracy(actual_labels, pred_gnb, pred_time, k=3):
    actual_labels_size = len(actual_labels['target_gnb'])
    predictions_size = len(pred_gnb)
    if actual_labels_size != predictions_size:
        raise ValueError("actual and predicted should be of same size")
    results = np.zeros(actual_labels_size)
    for i in range(actual_labels_size):
        if ((actual_labels['target_gnb'][i] in set(pred_gnb[i][:k])) & (actual_labels['target_trans_slot'][i] in set(pred_time[i][:k]))):
            results[i] = 1
    result = results.sum()/actual_labels_size
    print(f"predictionAccuracy with first {k} results: {result}")
    return result

In [ ]:
y_pred = model.predict(X_train)

In [ ]:
accuracy = compute_accuracy(y_train, y_pred)
accuracy

In [ ]:
probs = model.predict_proba(X_train)

In [ ]:
gnb_preds = model.classes_[0][np.argsort(-probs[0])]
ts_preds = model.classes_[1][np.argsort(-probs[1])]

In [ ]:
topk_accuracy(y_train.reset_index(), gnb_preds, ts_preds, k=1)
topk_accuracy(y_train.reset_index(), gnb_preds, ts_preds, k=3)
topk_accuracy(y_train.reset_index(), gnb_preds, ts_preds, k=5)

The following class implements the very same data processing procedure used above. It's been compacted for convenience

In [ ]:
class PreprocessData:
    def __init__(self, training_df,gnb_neigh):
        self.training_df = training_df
        self.gnb_neigh = gnb_neigh
    
    @staticmethod
    def window_sequence(seq, window_size=5):
        windows = []
        seq_len = len(seq)
        if seq_len < window_size:
            print(seq)
            padding_len = window_size - seq_len
        for i in range(len(seq) - window_size + 1):
            window = []
            for j in range(i, i + window_size):
                window.append(seq[j])
            windows.append(window)
        return windows

    def compute_home_gnb(self, grouped_df):
        ue_home_df = grouped_df.groupby(['imsi', 'currentEnb'])['stay_time'].sum().reset_index()
        home_gnb = ue_home_df.loc[ue_home_df.groupby(['imsi'])['stay_time'].idxmax()][['imsi', 'currentEnb']]
        home_gnb.columns = ['imsi', 'home_gnb']
        return home_gnb

    def compute_ue_context_df(self,grouped_df):
        grouped_df['time_of_day'] = pd.cut(grouped_df['timeSlot'], 
                                           bins=5,
                                           labels=['early_morn', 'morning', 'noon', 'evening', 'night'])
        ue_df = grouped_df.groupby(['imsi', 'time_of_day'])['currentEnb'].count().reset_index()
        ue_ctxt_df = pd.pivot_table(ue_df, index='imsi', columns='time_of_day', values='currentEnb', fill_value=0)
        return ue_ctxt_df

    def compute_transition_df(self, grouped_df):
        seq_df = grouped_df.groupby(['imsi'])['currentEnb', 'timeSlot','transition_slot'].agg(
            lambda x: list(x)).reset_index()
        seq_df['enode_seq'] = seq_df['currentEnb'].apply(self.window_sequence).values
        seq_df['time_seq'] = seq_df['timeSlot'].apply(self.window_sequence).values
        seq_df['trans_seq'] = seq_df['transition_slot'].apply(self.window_sequence).values
        enode_seq_list = [sequence for sequences in seq_df['enode_seq'] for sequence in sequences]
        time_seq_list = [sequence for sequences in seq_df['time_seq'] for sequence in sequences]
        trans_seq_list = [sequence for sequences in seq_df['trans_seq'] for sequence in sequences]
        seq_df['seq_len'] = seq_df['enode_seq'].apply(lambda x: len(x))
        imsi_list = list()
        for a, b in zip(seq_df['imsi'], seq_df['seq_len']):
            imsi_list.extend([a] * b)
        enode_df = pd.DataFrame(enode_seq_list, columns=['enode_1', 'enode_2', 'enode_3', 'enode_4', 'target_gnb'])
        time_df = pd.DataFrame(time_seq_list, columns=['time_1', 'time_2', 'time_3', 'time_4', 'target_time'])
        trans_df = pd.DataFrame(trans_seq_list, columns=['trans_1', 'trans_2', 'trans_3', 'trans_4', 'target_trans_slot'])
        df = pd.concat([enode_df, time_df], axis=1)
        #df = pd.concat([enode_df, time_df], axis=1)
        df['target_trans_slot'] = trans_df['target_trans_slot']
        df['imsi'] = imsi_list
        return df

    def preprocess_data(self):
        grouped_df = self.training_df.set_index(['imsi', 'timestamp']).sort_index().reset_index()
        grouped_df['next_timstamp'] = grouped_df.groupby(['imsi'])['timestamp'].shift(-1)
        grouped_df['prev_timstamp'] = grouped_df.groupby(['imsi'])['timestamp'].shift(1)
        grouped_df['stay_time'] = grouped_df['next_timstamp'] - grouped_df['timestamp']
        grouped_df['transit_time'] = grouped_df['timestamp'] - grouped_df['prev_timstamp']
        grouped_df['transition_slot'] = pd.cut(grouped_df['transit_time'], [0, 1, 5, 15, 30, 60, 3000],
                                               labels=[1, 5, 15, 30, 60, 100])

        home_gnb = self.compute_home_gnb(grouped_df)
        ue_ctxt_df = self.compute_ue_context_df(grouped_df)

        df = self.compute_transition_df(grouped_df)

        df = df.merge(home_gnb, on='imsi').merge(ue_ctxt_df, on='imsi')
        df = df.merge(self.gnb_neigh, left_on='enode_4', right_on='gnodeB', how='left')
        del df['gnodeB']
        df.dropna(inplace=True)
        y_train = df[['target_gnb', 'target_trans_slot']]

        to_drop_cols = ['target_gnb',
                        'target_trans_slot',
                        'target_time',
                        'imsi'
                       ]
        print(df.shape)
        df.drop(columns=to_drop_cols, inplace=True, axis=1)
        
        return df, y_train

In [ ]:
day_2 = df.loc[(df['timestamp']>=1440) & (df['timestamp']<2880)]

preprocess_df = PreprocessData(day_2,gnb_neigh)
X_test, y_test = preprocess_df.preprocess_data()
y_pred = model.predict(X_test)

In [ ]:
probs = model.predict_proba(X_test)
gnb_preds = model.classes_[0][np.argsort(-probs[0])]
ts_preds = model.classes_[1][np.argsort(-probs[1])]

In [ ]:
topk_accuracy(y_test.reset_index(), gnb_preds, ts_preds, k=1)
topk_accuracy(y_test.reset_index(), gnb_preds, ts_preds, k=3)
topk_accuracy(y_test.reset_index(), gnb_preds, ts_preds, k=5)

In [ ]:
accuracy = compute_accuracy(y_test, y_pred)
accuracy

## Model sharing
### Pickle file

In [ ]:
import joblib

joblib.dump(model, 'mobility_model.pkl')

### ONNX

Open Neural Network Exchange (ONNX) is an open standard format for AI models, born initially with focus on deep learning, but eventually extended to traditional ML. It defines an extensible computation graph model, as well as definitions of built-in operators and standard data types so that it is possible to represent any arbitrary model.

It is possible for most of the popular ML frameworks to export trained models as ONNX models. For scikit-learn: [sklearn-onnx](http://onnx.ai/sklearn-onnx/). The converted ONNX model can be run by means of the [ONNX Runtime](https://www.onnxruntime.ai/) (thus enabling the creation of microservices).

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import Int64TensorType

In [ ]:
X_train_onnx = X_train.to_numpy()
X_test_onnx = X_test.to_numpy()
y_train_onnx = y_train.to_numpy()
y_test_onnx = y_test.to_numpy()

In [ ]:
initial_type = [('input', Int64TensorType([None, 18]))]
onx = convert_sklearn(model, initial_types=initial_type, options={type(model): {'zipmap': False}})

In [ ]:
with open("mobility_model.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [ ]:
import onnxruntime as rt

In [ ]:
sess = rt.InferenceSession("mobility_model.onnx")

In [ ]:
input_name = sess.get_inputs()[0].name

In [ ]:
label_name = sess.get_outputs()[0].name

In [ ]:
sess.run([label_name], {input_name: X_test_onnx[:1]})

In [ ]:
model.predict(X_test_onnx[:1])